In [1]:
# --- PASO 0: REQUISITOS PREVIOS PARA GOOGLE COLAB ---
# ¡IMPORTANTE!: Este bloque asegura que Google Chrome y las librerías necesarias estén instaladas.

# Instalar Selenium, BeautifulSoup, Pandas y Webdriver Manager
!pip install selenium beautifulsoup4 pandas webdriver-manager requests dash > /dev/null 2>&1

# --- INSTALACIÓN DE GOOGLE CHROME EN COLAB ---
# 1. Descargar la clave GPG de Google Chrome
!wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | sudo gpg --dearmor -o /usr/share/keyrings/google-chrome-archive-keyring.gpg

# 2. Añadir el repositorio de Google Chrome a las fuentes de apt
!echo "deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main" | sudo tee /etc/apt/sources.list.d/google-chrome.list

# 3. Actualizar los listados de paquetes apt para incluir el nuevo repositorio
!sudo apt-get update

# 4. Instalar Google Chrome estable
!sudo apt-get install -y google-chrome-stable

# Opcional: Verificar la versión de Chrome instalada
!google-chrome --version

print("\n--- Instalación de Chrome y dependencias completada ---")


# --- PASO 2: IMPORTAR LIBRERÍAS ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

from webdriver_manager.chrome import ChromeDriverManager
import requests # Agregado para el scraping de Entel si usa requests

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import dash_table # Importar dash_table para la tabla de datos

# --- TUS FUNCIONES REALES DE EXTRACCIÓN DE PLANES ---
# ¡IMPORTANTE!: Estas funciones contienen el código de scraping para cada operador.
# Asegúrate de que todas devuelvan una lista de diccionarios con un esquema consistente.

def extraer_planes_movistar_colab():
    """
    Función para extraer planes de Movistar.
    Este código usa Selenium y BeautifulSoup para el scraping.
    Retorna una lista de diccionarios con los datos de los planes en formato estandarizado.
    """
    url = "https://www.movistar.com.pe/movil/postpago/planes-postpago"
    planes_data = []

    # --- CONFIGURACIÓN DE SELENIUM PARA COLAB ---
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--log-level=3')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = None

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # --- ESPERAR A QUE EL CONTENIDO DINÁMICO CARGUE ---
        wait = WebDriverWait(driver, 30)
        try:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'p-plan__slide__soles')))
        except Exception as e:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            try:
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'p-plan__slide__soles')))
            except Exception as e:
                print(f"ERROR (Movistar): 'p-plan__slide__soles' no se hizo visible incluso después de scroll. {e}")
                print("La estructura de la página o la forma en que carga el contenido ha cambiado significativamente o hay un bloqueo persistente.")
                return []

        time.sleep(5)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        plan_elements = soup.find_all('div', class_='p-plan__slide__shadow')

        if not plan_elements:
            print("ERROR (Movistar): No se encontraron elementos con la clase 'p-plan__slide__shadow'.")
            print("Esto podría indicar que la clase ha cambiado nuevamente o el contenido no se cargó como se esperaba.")
            return []

        for i, plan_element in enumerate(plan_elements):
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            detalles_gigas = 'N/A'
            apps_ilimitadas = 'No especificado'
            minutos_llamadas = 'No especificado'
            sms = 'No especificado'
            tipo_contratacion = 'Postpago'
            vigencia_promocion = 'N/A'

            precio_tag = plan_element.find('span', class_='p-plan__slide__soles')
            if precio_tag:
                precio = precio_tag.get_text(strip=True)

            head_tag = plan_element.find('div', class_='p-plan__slide__head')
            if head_tag:
                name_tag = head_tag.find(['h3', 'h4', 'span', 'p'], class_=lambda x: x and ('p-plan__slide__name' in x or 'title' in x or 'plan-name' in x))
                if name_tag:
                    nombre_plan = name_tag.get_text(strip=True)
                else:
                    nombre_plan = ' '.join(head_tag.get_text(separator=' ', strip=True).split())

                nombre_plan = re.sub(r'Plan Postpago\s*', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'S/\s*\d+\.\d+', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'al mes', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'x \d+ meses', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'Precio regular:.*?(Ahorra \d+%)?', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'Bono \d+ GB', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'Exclusivo online', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'\*', '', nombre_plan).strip()
                nombre_plan = re.sub(r'\s{2,}', ' ', nombre_plan).strip()

                if not nombre_plan:
                    nombre_plan = f"Plan Postpago {precio}" if precio != 'N/A' else 'Plan Postpago Desconocido'

            gigas_cantidad_tag = plan_element.find('span', class_='p-plan__slide__cantidad')
            if gigas_cantidad_tag:
                extracted_gigas = gigas_cantidad_tag.get_text(strip=True).replace('\n', ' ').strip()
                if "GB" in extracted_gigas.upper() or re.match(r'^\d+(\.\d+)?\s*GB$', extracted_gigas, re.IGNORECASE):
                    gigas = extracted_gigas
                    detalles_gigas = "En Alta Velocidad"
                elif "Bono" in extracted_gigas and "GB" in extracted_gigas:
                    gigas = extracted_gigas
                    detalles_gigas = "Bono de Gigas"

            if gigas == 'N/A':
                ilimitado_tag = plan_element.find(['span', 'div', 'h3', 'p'], class_=lambda x: x and ('p-plan__slide__gigas' in x or 'gigas-text' in x or 'data-info' in x))
                if ilimitado_tag and ("ilimitado" in ilimitado_tag.get_text().lower() or "sin límites" in ilimitado_tag.get_text().lower()):
                    gigas = "Ilimitados"
                    detalles_gigas = "Datos ilimitados (velocidad puede reducirse)"
                else:
                    text_content_lower = plan_element.get_text(separator=' ', strip=True).lower()
                    if "ilimitado" in text_content_lower and ("datos" in text_content_lower or "gigas" in text_content_lower):
                        gigas = "Ilimitados"
                        detalles_gigas = "Datos ilimitados (velocidad puede reducirse)"
                    else:
                        match_gb = re.search(r'(\d+)\s*gb', text_content_lower)
                        if match_gb:
                            gigas = f"{match_gb.group(1)} GB"
                            detalles_gigas = "En Alta Velocidad"
                        else:
                            match_bono_gb = re.search(r'(bono\s*\d+\s*gb\s*x\s*\d+\s*meses)', text_content_lower)
                            if match_bono_gb:
                                gigas = match_bono_gb.group(1).replace('x', 'x ')
                                detalles_gigas = "Bono de Gigas"

            apps_ttl_tag = plan_element.find('p', class_='p-plan__slide__apps__ttl')
            if apps_ttl_tag:
                apps_ilimitadas = apps_ttl_tag.get_text(strip=True)
                apps_ilimitadas = re.sub(r'\s*\n\s*', ' ', apps_ilimitadas).strip()
            else:
                apps_ilimitadas_list_temp = []
                apps_container = plan_element.find('div', class_='p-plan__slide__apps')
                if apps_container:
                    app_elements = apps_container.find_all(['img', 'span', 'i', 'p'], class_=lambda x: x and ('app-icon' in x or 'unlimited-app-icon' in x or 'logo-app' in x or 'app-name' in x))
                    for app_el in app_elements:
                        app_name = app_el.get('alt') or app_el.get('title') or app_el.get_text(strip=True)
                        if app_name and app_name.strip():
                            apps_ilimitadas_list_temp.append(re.sub(r'\s*\n\s*', ' ', app_name).strip())

                if not apps_ilimitadas_list_temp:
                    text_content_full = plan_element.get_text(separator=' ', strip=True)
                    match_apps_text = re.search(r'(?:Apps|Redes Sociales)\s+Ilimitadas(?::\s*(.*?))?(?=[.\n]|$)', text_content_full, re.IGNORECASE | re.DOTALL)
                    if match_apps_text:
                        if match_apps_text.group(1):
                            apps_ilimitadas_list_temp.extend([re.sub(r'\s*\n\s*', ' ', app.strip()).strip() for app in match_apps_text.group(1).split(',') if app.strip()])
                        else:
                            apps_ilimitadas_list_temp.append(re.sub(r'\s*\n\s*', ' ', match_apps_text.group(0).replace(":", "").strip()).strip())

                    keywords = ["WhatsApp", "Facebook", "Instagram", "TikTok", "Spotify", "Netflix", "Youtube", "Waze", "Twitter"]
                    for keyword in keywords:
                        if f"{keyword} Ilimitado" in text_content_full or f"Acceso ilimitado a {keyword}" in text_content_full:
                            if keyword.lower() not in [a.lower() for a in apps_ilimitadas_list_temp]:
                                apps_ilimitadas_list_temp.append(keyword)

                if apps_ilimitadas_list_temp:
                    apps_ilimitadas = ", ".join(sorted(list(set(apps_ilimitadas_list_temp))))

            minutos_llamadas = 'No especificado'
            sms = 'No especificado'

            all_benefit_texts_raw = []
            benefit_text_tags_specific = plan_element.find_all('p', class_='stefa-parrilla_blanco--body-texto')
            for tag in benefit_text_tags_specific:
                all_benefit_texts_raw.append(tag.get_text(strip=True))

            details_container = plan_element.find('div', class_='p-plan__slide__details')
            if details_container:
                general_benefit_tags = details_container.find_all(['li', 'p', 'span', 'div'], class_=lambda x: x and ('benefit-item' in x or 'feature-row' in x or 'text-benefit' in x or 'item-detail' in x or 'body-text' in x or 'plan-detail' in x))
                for tag in general_benefit_tags:
                    all_benefit_texts_raw.append(tag.get_text(strip=True))

            full_plan_text = plan_element.get_text(separator=' ', strip=True)
            all_benefit_texts_raw.append(full_plan_text)

            processed_texts = set()

            for raw_text in all_benefit_texts_raw:
                cleaned_text = re.sub(r'\s*\n\s*', ' ', raw_text).strip()
                if not cleaned_text or cleaned_text in processed_texts:
                    continue
                processed_texts.add(cleaned_text)

                text_lower = cleaned_text.lower()

                if 'llamadas ilimitadas' in text_lower:
                    minutos_llamadas = 'Llamadas ilimitadas'
                    match_usa_canada = re.search(r'(\d+)\s*minutos\s*(?:para|a)\s*(?:usa|eeuu)\s*(?:y|e)\s*canadá', text_lower)
                    if match_usa_canada:
                        minutos_llamadas += f", {match_usa_canada.group(1)} minutos para Usa y Canadá"
                elif re.search(r'(\d+)\s*minutos', text_lower):
                    match_minutes = re.search(r'(\d+)\s*minutos', text_lower)
                    if match_minutes:
                        minutos_llamadas = f"{match_minutes.group(1)} minutos"

                if 'sms ilimitados' in text_lower:
                    sms = 'SMS ilimitados'
                elif re.search(r'(\d+)\s*sms', text_lower):
                    match_sms = re.search(r'(\d+)\s*sms', text_lower)
                    if match_sms:
                        sms = f"{match_sms.group(1)} SMS"

            if "internet + llamadas ilimitadas" in apps_ilimitadas.lower() and minutos_llamadas == 'No especificado':
                minutos_llamadas = "Llamadas ilimitadas"
                apps_ilimitadas = apps_ilimitadas.replace("Internet + Llamadas Ilimitadas", "Internet Ilimitado (si aplica)").strip()
                if not apps_ilimitadas or apps_ilimitadas.lower() == "internet ilimitado (si aplica)".lower():
                    apps_ilimitadas = 'No especificado'

            planes_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Detalles de Gigas': detalles_gigas,
                'Apps Ilimitadas': apps_ilimitadas,
                'Minutos/Llamadas': minutos_llamadas,
                'SMS': sms,
                'Tipo de Contratación': tipo_contratacion,
                'Vigencia de Promoción': vigencia_promocion
            })

    except Exception as e:
        print(f"Ocurrió un error crítico durante la extracción de Movistar: {e}")
        return []
    finally:
        if driver:
            driver.quit()

    return planes_data

def extraer_planes_claro_colab():
    """
    Función para extraer planes de Claro.
    Este código usa Selenium y BeautifulSoup para el scraping.
    Retorna una lista de diccionarios con los datos de los planes en formato estandarizado.
    """
    url = "https://www.claro.com.pe/personas/movil/postpago/"
    plans_data = []
    processed_plans = set()

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--log-level=3')

    driver = None

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'cA1PEBodyCardWrap')))
        time.sleep(5)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        plan_elements = soup.find_all('div', class_='cA1PEBodyCardWrap')

        if not plan_elements:
            print("ERROR (Claro): No se encontraron elementos de planes en la página. La estructura HTML podría haber cambiado.")
            return []

        for plan_element in plan_elements:
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            detalles_gigas = 'N/A'
            apps_ilimitadas = 'No especificado'
            minutos_llamadas = 'No especificado'
            sms = 'No especificado'
            tipo_contratacion = 'Postpago'
            vigencia_promocion = 'N/A'

            name = plan_element.get('data-badge', 'N/A')
            nombre_plan = name

            price_str = plan_element.get('data-price', 'N/A')
            try:
                precio = float(price_str)
            except ValueError:
                precio = 'N/A'

            plan_key = (nombre_plan, precio)
            if plan_key in processed_plans:
                continue
            processed_plans.add(plan_key)

            gigas_tag = plan_element.find('span', class_='number')
            if gigas_tag:
                gigas_text = gigas_tag.get_text(strip=True)
                if gigas_text.upper().endswith('GB'):
                    gigas = gigas_text
                    if "ilimitado" in plan_element.get_text().lower() or "velocidad reducida" in plan_element.get_text().lower():
                        detalles_gigas = f"{gigas} en Alta Velocidad (luego velocidad reducida)"
                    else:
                        detalles_gigas = "En Alta Velocidad"
                elif "ilimitado" in gigas_text.lower():
                    gigas = "Ilimitados"
                    detalles_gigas = "Datos ilimitados (velocidad puede reducirse)"
                else:
                    gigas = gigas_text
                    detalles_gigas = "N/A"

            is_max_ilimitado_promo = False
            promo_div = plan_element.find('div', class_='cardPePromo')
            if promo_div:
                promo_text_span = promo_div.find('span', string=lambda text: text and 'Gigas, Minutos y SMS' in text)
                if promo_text_span:
                    is_max_ilimitado_promo = True
                    minutos_llamadas = "Ilimitadas"
                    sms = "Ilimitados"
                    apps_ilimitadas = "Incluidas en Todo Ilimitado"

            if not is_max_ilimitado_promo:
                apps_list = []
                app_icon_tags = plan_element.find_all('i', class_=lambda x: x and 'cIco-rs-' in x)
                for icon_tag in app_icon_tags:
                    for cls in icon_tag.get('class', []):
                        if 'cIco-rs-' in cls:
                            app_name = cls.replace('cIco-rs-', '')
                            apps_list.append(app_name.capitalize())
                apps_ilimitadas = ", ".join(apps_list) if apps_list else 'No especificado'
                if apps_list:
                    apps_ilimitadas = f"{apps_ilimitadas} ilimitadas"

                if minutos_llamadas == 'No especificado':
                    span_element_with_text = plan_element.find('span', string=lambda text: text and ('Llamadas y SMS' in text.strip() or 'Llamadas' in text.strip() or 'SMS' in text.strip()))
                    if span_element_with_text:
                        dt_parent = span_element_with_text.find_parent('dt')
                        if dt_parent:
                            dd_element = dt_parent.find_next_sibling('dd')
                            if dd_element:
                                calls_sms_text = dd_element.get_text(strip=True)
                                if "ilimitadas" in calls_sms_text.lower():
                                    minutos_llamadas = "Llamadas ilimitadas"
                                    sms = "SMS ilimitados"
                                else:
                                    match_minutes = re.search(r'(\d+)\s*minutos', calls_sms_text, re.IGNORECASE)
                                    if match_minutes:
                                        minutos_llamadas = f"{match_minutes.group(1)} minutos"
                                    else:
                                        minutos_llamadas = calls_sms_text
                                    match_sms = re.search(r'(\d+)\s*sms', calls_sms_text, re.IGNORECASE)
                                    if match_sms:
                                        sms = f"{match_sms.group(1)} SMS"
                                    else:
                                        sms = calls_sms_text

            plans_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Detalles de Gigas': detalles_gigas,
                'Apps Ilimitadas': apps_ilimitadas,
                'Minutos/Llamadas': minutos_llamadas,
                'SMS': sms,
                'Tipo de Contratacion': tipo_contratacion,
                'Vigencia de Promoción': vigencia_promocion
            })

    except Exception as e:
        print(f"Ocurrió un error crítico durante la extracción de Claro: {e}")
        return []
    finally:
        if driver:
            driver.quit()

    return plans_data

def extraer_planes_bitel_colab():
    """
    Función para extraer planes de Bitel.
    Este código usa Selenium y BeautifulSoup para el scraping.
    Retorna una lista de diccionarios con los datos de los planes.
    """
    url = "https://bitel.com.pe/planes/control/ilimitado"
    planes_data = []

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1990,1080')
    options.add_argument('--log-level=3')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = None
    wait = None

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        wait = WebDriverWait(driver, 45)

        try:
            common_close_xpaths = [
                "//button[contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'Cerrar') or contains(., 'OK')]",
                "//a[contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'Cerrar') or contains(., 'OK')]",
                "//div[contains(@class, 'close-button') or contains(@class, 'modal-close') or contains(@class, 'btn-close') or contains(@class, 'close-popup')]",
                "//span[contains(text(), 'x') or contains(text(), 'X') or @class='close-icon']",
                "//button[contains(@id, 'cookie') or contains(@id, 'modal') or contains(@class, 'cookie') or contains(@class, 'modal')][contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'OK')]",
                "//div[@role='dialog']//button[contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'OK')]"
            ]

            found_and_clicked = False
            for xpath_str in common_close_xpaths:
                try:
                    btn = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, xpath_str))
                    )
                    if btn.is_displayed() and btn.is_enabled():
                        btn.click()
                        time.sleep(2)
                        found_and_clicked = True
                        break
                except:
                    pass

        except Exception as e:
            pass

        try:
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'cont-package')))
        except Exception as e:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            try:
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'cont-package')))
            except Exception as e:
                print(f"ERROR FATAL: 'cont-package' no se hizo visible incluso después de scroll. {e}")
                print("La estructura de la página o la forma en que carga el contenido ha cambiado significativamente o hay un bloqueo persistente.")
                return []

        time.sleep(5)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        plan_elements = soup.find_all('div', class_='cont-package')

        if not plan_elements:
            print("ERROR: Después de la carga, no se encontraron elementos con la clase 'cont-package' en BeautifulSoup.")
            print("Esto podría indicar que la clase ha cambiado o el contenido no está en el HTML parseado.")
            return []

        for i, plan_element in enumerate(plan_elements):
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            apps_ilimitadas = 'No especificado'
            minutos_llamadas = 'No especificado'
            sms = 'No especificado'
            detalles_gigas = 'N/A'

            full_plan_text_raw = plan_element.get_text(separator=' ', strip=True)
            full_plan_text_lower = full_plan_text_raw.lower()

            match_precio = re.search(r's/\s*(\d+\.\d+)', full_plan_text_lower)
            if match_precio:
                precio = f"S/ {match_precio.group(1)}"
            else:
                price_element_a = plan_element.find('a', href="javascript:void(0);")
                if price_element_a:
                    title_attr = price_element_a.get('title')
                    if title_attr:
                        match_price_title = re.search(r'(\d+\.\d+)', title_attr)
                        if match_price_title:
                            precio = f"S/ {match_price_title.group(1)}"

                if precio == 'N/A':
                    match_precio_simple = re.search(r'(\d+\.\d+)', full_plan_text_lower)
                    if match_precio_simple:
                        precio_val = float(match_precio_simple.group(1))
                        if 10.00 <= precio_val <= 200.00:
                            precio = f"S/ {match_precio_simple.group(1)}"

            if precio != 'N/A':
                nombre_plan = f"Ilimitado - {precio.replace('S/ ', '')}"
            else:
                name_span_element = plan_element.find('span', class_='color-white text-bold title-1g')
                if name_span_element:
                    span_text = name_span_element.get_text(strip=True)
                    cleaned_span_text = re.sub(r'\s*S/\s*\d+\.\d+', '', span_text, flags=re.IGNORECASE).strip()
                    cleaned_span_text = re.sub(r'\s*\d+\.\d+$', '', cleaned_span_text, flags=re.IGNORECASE).strip()
                    if cleaned_span_text:
                        nombre_plan = f"Ilimitado - {cleaned_span_text}"
                    else:
                        nombre_plan = "Ilimitado - Precio No Disponible"
                else:
                    nombre_plan = "Ilimitado - Precio No Disponible"

            gigas_element = plan_element.find('p', class_='capa')

            if gigas_element:
                gigas_text = gigas_element.get_text(strip=True)
                gigas_text_lower = gigas_text.lower()

                period_element = plan_element.find('p', class_='period')
                period_text_lower = period_element.get_text(strip=True).lower() if period_element else ''

                if 'ilimitados' in gigas_text_lower:
                    gigas = 'Ilimitados'
                    match_gigas_alta_velocidad = re.search(r'(\d+)\s*GB(?:\s*en\s*alta\s*velocidad)?', gigas_text, re.IGNORECASE)
                    if match_gigas_alta_velocidad:
                        detalles_gigas = f"{match_gigas_alta_velocidad.group(1)} GB en Alta Velocidad"
                    elif 'alta velocidad' in period_text_lower:
                        detalles_gigas = "Velocidad reducida después de cierto consumo"
                    else:
                        detalles_gigas = "Datos ilimitados sin restricciones de velocidad explícitas"
                else:
                    match_gigas = re.search(r'(\d+)\s*GB', gigas_text, re.IGNORECASE)
                    if match_gigas:
                        gigas = f"{match_gigas.group(1)} GB"
                        if 'alta velocidad' in period_text_lower:
                            detalles_gigas = "En Alta Velocidad"
                    else:
                        gigas = gigas_text
                        if 'alta velocidad' in period_text_lower:
                            detalles_gigas = "En Alta Velocidad"
            else:
                gigas = 'N/A'
                detalles_gigas = 'Elemento de gigas no encontrado'

            app_keywords_patterns = {
                'WhatsApp': r'whatsapp ilimitado',
                'Facebook': r'facebook ilimitado',
                'Instagram': r'instagram ilimitado',
                'TikTok': r'tiktok ilimitado',
                'Spotify': r'spotify ilimitado',
                'Waze': r'waze ilimitado',
                'YouTube': r'youtube ilimitado',
                'Apps ilimitadas x meses': r'apps ilimitadas (x \d+ meses)?',
                'Internet + Llamadas ilimitadas': r'internet \+\s*llamadas ilimitadas'
            }
            found_apps = []
            for app_name, pattern in app_keywords_patterns.items():
                if re.search(pattern, full_plan_text_lower):
                    if 'x \d+ meses' in pattern and re.search(r'x \d+ meses', full_plan_text_lower):
                        promo_match = re.search(r'x \d+ meses', full_plan_text_lower).group(0)
                        found_apps.append(f"{app_name.replace(' x meses', '')} {promo_match}")
                    else:
                        found_apps.append(app_name)

            app_image_elements = plan_element.find_all('li', class_='app-item')
            for app_li in app_image_elements:
                img_tag = app_li.find('img')
                if img_tag and 'alt' in img_tag.attrs:
                    app_name_from_alt = img_tag['alt'].strip()
                    if app_name_from_alt and app_name_from_alt.lower() not in [app.lower() for app in found_apps]:
                        found_apps.append(app_name_from_alt)

            apps_ilimitadas = ", ".join(sorted(list(set(found_apps)))) if found_apps else 'No especificado'

            minutos_llamadas = "No especificado"
            sms = "No especificado"

            todo_ilimitado_element = plan_element.find('p', class_=re.compile(r'title.*Todo ilimitado'))
            if todo_ilimitado_element and "todo ilimitado" in todo_ilimitado_element.get_text(strip=True).lower():
                minutos_llamadas = 'Llamadas ilimitadas'
                sms = 'SMS ilimitados'
            else:
                if 'llamadas ilimitadas perú' in full_plan_text_lower:
                    minutos_llamadas = 'Llamadas ilimitadas Perú'
                    match_usa_canada = re.search(r'(\d+)\s*minutos\s*(?:para|a)\s*(?:usa|eeuu)\s*(?:y|e)\s*canadá', full_plan_text_lower)
                    if match_usa_canada:
                        minutos_llamadas += f", {match_usa_canada.group(1)} minutos para Usa y Canadá"
                elif 'llamadas ilimitadas' in full_plan_text_lower:
                    minutos_llamadas = 'Llamadas ilimitadas'

                if 'sms ilimitados' in full_plan_text_lower:
                    sms = 'SMS ilimitados'
                else:
                    match_sms = re.search(r'(\d+)\s*sms', full_plan_text_lower)
                    if match_sms:
                        sms = f"{match_sms.group(1)} SMS"

            benefit_period_element = plan_element.find('p', class_='benefit-period')
            if benefit_period_element and "internet, llamadas y sms" in benefit_period_element.get_text(strip=True).lower():
                if minutos_llamadas == "No especificado":
                    minutos_llamadas = 'Llamadas incluidas'
                if sms == "No especificado":
                    sms = 'SMS incluidos'

            planes_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Detalles de Gigas': detalles_gigas,
                'Apps Ilimitadas': apps_ilimitadas,
                'Minutos/Llamadas': minutos_llamadas,
                'SMS': sms,
                'Tipo de Contratación': 'Postpago',
                'Vigencia de Promoción': 'N/A'
            })

    except Exception as e:
        print(f"Ocurrió un error crítico durante la ejecución: {e}")
        return []
    finally:
        if driver:
            driver.quit()

    return planes_data

def extraer_planes_entel_colab():
    """
    Función para extraer planes de Entel.
    Este código usa requests y BeautifulSoup para el scraping.
    Retorna una lista de diccionarios con los datos de los planes en formato estandarizado.
    """
    url = "https://ofertasentel.pe/?utm_source=bing&utm_medium=cpc_search&utm_campaign=pospago_promo_marcaplanes&utm_term=marca_planes&utmcampaign=0104020302&msclkid=dfab6aa18cde18a17c3b02cd920ff43b"
    planes_data = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        plan_boxes = soup.find_all('div', class_='box')

        if not plan_boxes:
            print("ERROR (Entel): No se encontraron elementos con la clase 'box' en BeautifulSoup.")
            return []

        for box in plan_boxes:
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            detalles_gigas = 'N/A'
            apps_ilimitadas = 'No especificado'
            minutos_llamadas = 'No especificado'
            sms = 'No especificado'
            tipo_contratacion = 'Postpago'
            vigencia_promocion = 'N/A'

            plan_name_tag = box.find('div', class_='property property_internet')
            if plan_name_tag and plan_name_tag.find('b'):
                nombre_plan = plan_name_tag.find('b').get_text(strip=True)
            elif box.find('h3'):
                nombre_plan = box.find('h3').get_text(strip=True)

            price_tag = box.find('div', class_='price')
            if price_tag:
                precio = price_tag.get_text(strip=True)

            gb_div = box.find('div', class_='gb')
            if gb_div:
                strong_text = gb_div.find('strong').get_text(strip=True) if gb_div.find('strong') else ''
                span_text = gb_div.find('span').get_text(strip=True) if gb_div.find('span') else ''

                if 'ilimitado' in strong_text.lower():
                    gigas = 'Ilimitados'
                    if span_text:
                        detalles_gigas = span_text.strip()
                    else:
                        detalles_gigas = "Datos ilimitados sin restricciones de velocidad explícitas"
                else:
                    gigas = strong_text.strip()
                    if span_text:
                        detalles_gigas = span_text.strip()
                    else:
                        detalles_gigas = "N/A"

            app_tag = box.find('div', class_='property property_app')
            if app_tag:
                app_list_items = app_tag.find_all('li')
                if app_list_items:
                    apps_ilimitadas = ", ".join(sorted(list(set([li.get_text(strip=True) for li in app_list_items if li.get_text(strip=True)]))))
                else:
                    apps_ilimitadas = app_tag.get_text(strip=True, separator=' ')
                if "ilimitado" not in apps_ilimitadas.lower() and "ilimitadas" not in apps_ilimitadas.lower() and apps_ilimitadas != 'N/A' and apps_ilimitadas != 'No especificado':
                     apps_ilimitadas = f"{apps_ilimitadas} ilimitadas"

            llamadas_tag = box.find('div', class_='property property_llamadas')
            if llamadas_tag:
                llamadas_text = llamadas_tag.get_text(strip=True)
                if "ilimitadas" in llamadas_text.lower():
                    minutos_llamadas = "Llamadas ilimitadas"
                else:
                    minutos_llamadas = llamadas_text

            sms_tag = box.find('div', class_='property property_sms')
            if sms_tag:
                sms_text = sms_tag.get_text(strip=True)
                if "ilimitados" in sms_text.lower():
                    sms = "SMS ilimitados"
                else:
                    sms = sms_text

            planes_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Detalles de Gigas': detalles_gigas,
                'Apps Ilimitadas': apps_ilimitadas,
                'Minutos/Llamadas': minutos_llamadas,
                'SMS': sms,
                'Tipo de Contratacion': tipo_contratacion,
                'Vigencia de Promoción': vigencia_promocion
            })

    except requests.exceptions.RequestException as e:
        print(f"Error de red al acceder a la página de Entel: {e}")
        return []
    except Exception as e:
        print(f"Ocurrió un error inesperado durante el scraping de Entel: {e}")
        return []

    return planes_data

def standardize_plan_data(plan):
    """
    Estandariza las claves y los tipos de datos de un diccionario de plan.
    Asegura que 'Precio (S/)' sea un float.
    """
    # Definir todas las claves esperadas y sus valores por defecto
    standardized_plan = {
        'Operador': plan.get('Operador', 'N/A'),
        'Nombre del Plan': plan.get('Nombre del Plan', 'N/A'),
        'Precio (S/)': plan.get('Precio (S/)', 'N/A'),
        'Gigas': plan.get('Gigas', 'N/A'),
        'Detalles de Gigas': plan.get('Detalles de Gigas', 'N/A'),
        'Apps Ilimitadas': plan.get('Apps Ilimitadas', 'No especificado'),
        'Minutos/Llamadas': plan.get('Minutos/Llamadas', 'No especificado'),
        'SMS': plan.get('SMS', 'No especificado'),
        'Tipo de Contratación': plan.get('Tipo de Contratación', 'Postpago'),
        'Vigencia de Promoción': plan.get('Vigencia de Promoción', 'N/A')
    }

    # Convertir Precio (S/) a float
    precio_raw = standardized_plan['Precio (S/)']
    if isinstance(precio_raw, str):
        precio_str = precio_raw.replace('S/', '').replace(',', '.').strip()
        try:
            standardized_plan['Precio (S/)'] = float(precio_str)
        except ValueError:
            standardized_plan['Precio (S/)'] = None # Usar None si no se puede convertir a número
    elif not isinstance(precio_raw, (int, float)):
        standardized_plan['Precio (S/)'] = None # Asegurarse de que sea None si no es número ni string convertible

    return standardized_plan

def get_all_telecom_plans():
    """
    Extrae y combina los planes de todos los operadores.
    Retorna un DataFrame de Pandas con los datos estandarizados.
    """
    all_plans = []

    # --- Extraer y combinar planes de Movistar ---
    try:
        movistar_plans = extraer_planes_movistar_colab()
        for plan in movistar_plans:
            plan['Operador'] = 'Movistar'
            all_plans.append(standardize_plan_data(plan))
    except Exception as e:
        print(f"Error al extraer planes de Movistar: {e}")

    # --- Extraer y combinar planes de Claro ---
    try:
        claro_plans = extraer_planes_claro_colab()
        for plan in claro_plans:
            plan['Operador'] = 'Claro'
            all_plans.append(standardize_plan_data(plan))
    except Exception as e:
        print(f"Error al extraer planes de Claro: {e}")

    # --- Extraer y combinar planes de Bitel ---
    try:
        bitel_plans = extraer_planes_bitel_colab()
        for plan in bitel_plans:
            plan['Operador'] = 'Bitel'
            all_plans.append(standardize_plan_data(plan))
    except Exception as e:
        print(f"Error al extraer planes de Bitel: {e}")

    # --- Extraer y combinar planes de Entel ---
    try:
        entel_plans = extraer_planes_entel_colab()
        for plan in entel_plans:
            plan['Operador'] = 'Entel'
            all_plans.append(standardize_plan_data(plan))
    except Exception as e:
        print(f"Error al extraer planes de Entel: {e}")

    df = pd.DataFrame(all_plans)

    # Ordenar por precio, asegurando que los valores 'N/A' o None vayan al final
    df['Precio_Sort'] = df['Precio (S/)'].apply(lambda x: x if isinstance(x, (int, float)) else float('inf'))
    df_sorted = df.sort_values(by='Precio_Sort').drop(columns='Precio_Sort')

    return df_sorted

# --- Inicializar la aplicación Dash ---
app = dash.Dash(__name__)

# Cargar los datos iniciales
df_plans = get_all_telecom_plans()

# Obtener opciones únicas de operadores para el dropdown
operator_options = [{'label': 'Todos', 'value': 'all'}] + \
                   [{'label': op, 'value': op} for op in df_plans['Operador'].unique()]

# --- Diseño de la aplicación Dash ---
app.layout = html.Div(style={'fontFamily': 'Arial, sans-serif', 'padding': '20px', 'backgroundColor': '#f4f4f4', 'color': '#333'}, children=[
    html.H1("Comparador de Planes Telefónicos", style={'textAlign': 'center', 'color': '#007bff'}),

    html.Div([
        html.Label("Filtrar por Operador:", style={'marginRight': '10px', 'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='operator-dropdown',
            options=operator_options,
            value='all',
            clearable=False,
            style={'width': '200px', 'display': 'inline-block', 'verticalAlign': 'middle'}
        ),
    ], style={'marginBottom': '20px', 'textAlign': 'center'}),

    dash_table.DataTable(
        id='plans-table',
        columns=[{"name": i, "id": i} for i in df_plans.columns],
        data=df_plans.to_dict('records'),
        style_table={'overflowX': 'auto', 'boxShadow': '0 4px 12px rgba(0,0,0,0.15)', 'borderRadius': '8px'},
        style_header={
            'backgroundColor': '#00AEEF',
            'color': 'white',
            'fontWeight': 'bold',
            'textAlign': 'left',
            'padding': '15px 20px',
            'textTransform': 'uppercase',
            'fontSize': '0.95em',
            'letterSpacing': '0.5px'
        },
        style_data={
            'backgroundColor': 'white',
            'color': '#333',
            'borderBottom': '1px solid #eee',
            'padding': '15px 20px'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'even'},
                'backgroundColor': '#f8f8f8'
            },
            {
                'if': {'state': 'active'},
                'backgroundColor': '#e0f7fa',
                'border': '1px solid #00AEEF'
            }
        ],
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_size=10,
    )
])

# --- Callbacks para interactividad ---
@app.callback(
    Output('plans-table', 'data'),
    Input('operator-dropdown', 'value')
)
def update_table(selected_operator):
    if selected_operator == 'all':
        return df_plans.to_dict('records')
    else:
        filtered_df = df_plans[df_plans['Operador'] == selected_operator]
        return filtered_df.to_dict('records')

# --- Ejecutar la aplicación Dash en Colab ---
if __name__ == '__main__':
    app.run_server(mode='inline', port=8050)


deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,215 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,092 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadc

ObsoleteAttributeException: app.run_server has been replaced by app.run

In [2]:
# --- PASO 0: REQUISITOS PREVIOS PARA GOOGLE COLAB ---
# ¡IMPORTANTE!: Este bloque asegura que Google Chrome y las librerías necesarias estén instaladas.

# Instalar Selenium, BeautifulSoup, Pandas y Webdriver Manager
!pip install selenium beautifulsoup4 pandas webdriver-manager requests dash > /dev/null 2>&1

# --- INSTALACIÓN DE GOOGLE CHROME EN COLAB ---
# 1. Descargar la clave GPG de Google Chrome
!wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | sudo gpg --dearmor -o /usr/share/keyrings/google-chrome-archive-keyring.gpg

# 2. Añadir el repositorio de Google Chrome a las fuentes de apt
!echo "deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main" | sudo tee /etc/apt/sources.list.d/google-chrome.list

# 3. Actualizar los listados de paquetes apt para incluir el nuevo repositorio
!sudo apt-get update

# 4. Instalar Google Chrome estable
!sudo apt-get install -y google-chrome-stable

# Opcional: Verificar la versión de Chrome instalada
!google-chrome --version

print("\n--- Instalación de Chrome y dependencias completada ---")


# --- PASO 2: IMPORTAR LIBRERÍAS ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

from webdriver_manager.chrome import ChromeDriverManager
import requests # Agregado para el scraping de Entel si usa requests

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import dash_table # Importar dash_table para la tabla de datos

# --- TUS FUNCIONES REALES DE EXTRACCIÓN DE PLANES ---
# ¡IMPORTANTE!: Estas funciones contienen el código de scraping para cada operador.
# Asegúrate de que todas devuelvan una lista de diccionarios con un esquema consistente.

def extraer_planes_movistar_colab():
    """
    Función para extraer planes de Movistar.
    Este código usa Selenium y BeautifulSoup para el scraping.
    Retorna una lista de diccionarios con los datos de los planes en formato estandarizado.
    """
    url = "https://www.movistar.com.pe/movil/postpago/planes-postpago"
    planes_data = []

    # --- CONFIGURACIÓN DE SELENIUM PARA COLAB ---
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--log-level=3')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = None

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # --- ESPERAR A QUE EL CONTENIDO DINÁMICO CARGUE ---
        wait = WebDriverWait(driver, 30)
        try:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'p-plan__slide__soles')))
        except Exception as e:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            try:
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'p-plan__slide__soles')))
            except Exception as e:
                print(f"ERROR (Movistar): 'p-plan__slide__soles' no se hizo visible incluso después de scroll. {e}")
                print("La estructura de la página o la forma en que carga el contenido ha cambiado significativamente o hay un bloqueo persistente.")
                return []

        time.sleep(5)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        plan_elements = soup.find_all('div', class_='p-plan__slide__shadow')

        if not plan_elements:
            print("ERROR (Movistar): No se encontraron elementos con la clase 'p-plan__slide__shadow'.")
            print("Esto podría indicar que la clase ha cambiado nuevamente o el contenido no se cargó como se esperaba.")
            return []

        for i, plan_element in enumerate(plan_elements):
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            detalles_gigas = 'N/A'
            apps_ilimitadas = 'No especificado'
            minutos_llamadas = 'No especificado'
            sms = 'No especificado'
            tipo_contratacion = 'Postpago'
            vigencia_promocion = 'N/A'

            precio_tag = plan_element.find('span', class_='p-plan__slide__soles')
            if precio_tag:
                precio = precio_tag.get_text(strip=True)

            head_tag = plan_element.find('div', class_='p-plan__slide__head')
            if head_tag:
                name_tag = head_tag.find(['h3', 'h4', 'span', 'p'], class_=lambda x: x and ('p-plan__slide__name' in x or 'title' in x or 'plan-name' in x))
                if name_tag:
                    nombre_plan = name_tag.get_text(strip=True)
                else:
                    nombre_plan = ' '.join(head_tag.get_text(separator=' ', strip=True).split())

                nombre_plan = re.sub(r'Plan Postpago\s*', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'S/\s*\d+\.\d+', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'al mes', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'x \d+ meses', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'Precio regular:.*?(Ahorra \d+%)?', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'Bono \d+ GB', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'Exclusivo online', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'\*', '', nombre_plan).strip()
                nombre_plan = re.sub(r'\s{2,}', ' ', nombre_plan).strip()

                if not nombre_plan:
                    nombre_plan = f"Plan Postpago {precio}" if precio != 'N/A' else 'Plan Postpago Desconocido'

            gigas_cantidad_tag = plan_element.find('span', class_='p-plan__slide__cantidad')
            if gigas_cantidad_tag:
                extracted_gigas = gigas_cantidad_tag.get_text(strip=True).replace('\n', ' ').strip()
                if "GB" in extracted_gigas.upper() or re.match(r'^\d+(\.\d+)?\s*GB$', extracted_gigas, re.IGNORECASE):
                    gigas = extracted_gigas
                    detalles_gigas = "En Alta Velocidad"
                elif "Bono" in extracted_gigas and "GB" in extracted_gigas:
                    gigas = extracted_gigas
                    detalles_gigas = "Bono de Gigas"

            if gigas == 'N/A':
                ilimitado_tag = plan_element.find(['span', 'div', 'h3', 'p'], class_=lambda x: x and ('p-plan__slide__gigas' in x or 'gigas-text' in x or 'data-info' in x))
                if ilimitado_tag and ("ilimitado" in ilimitado_tag.get_text().lower() or "sin límites" in ilimitado_tag.get_text().lower()):
                    gigas = "Ilimitados"
                    detalles_gigas = "Datos ilimitados (velocidad puede reducirse)"
                else:
                    text_content_lower = plan_element.get_text(separator=' ', strip=True).lower()
                    if "ilimitado" in text_content_lower and ("datos" in text_content_lower or "gigas" in text_content_lower):
                        gigas = "Ilimitados"
                        detalles_gigas = "Datos ilimitados (velocidad puede reducirse)"
                    else:
                        match_gb = re.search(r'(\d+)\s*gb', text_content_lower)
                        if match_gb:
                            gigas = f"{match_gb.group(1)} GB"
                            detalles_gigas = "En Alta Velocidad"
                        else:
                            match_bono_gb = re.search(r'(bono\s*\d+\s*gb\s*x\s*\d+\s*meses)', text_content_lower)
                            if match_bono_gb:
                                gigas = match_bono_gb.group(1).replace('x', 'x ')
                                detalles_gigas = "Bono de Gigas"

            apps_ttl_tag = plan_element.find('p', class_='p-plan__slide__apps__ttl')
            if apps_ttl_tag:
                apps_ilimitadas = apps_ttl_tag.get_text(strip=True)
                apps_ilimitadas = re.sub(r'\s*\n\s*', ' ', apps_ilimitadas).strip()
            else:
                apps_ilimitadas_list_temp = []
                apps_container = plan_element.find('div', class_='p-plan__slide__apps')
                if apps_container:
                    app_elements = apps_container.find_all(['img', 'span', 'i', 'p'], class_=lambda x: x and ('app-icon' in x or 'unlimited-app-icon' in x or 'logo-app' in x or 'app-name' in x))
                    for app_el in app_elements:
                        app_name = app_el.get('alt') or app_el.get('title') or app_el.get_text(strip=True)
                        if app_name and app_name.strip():
                            apps_ilimitadas_list_temp.append(re.sub(r'\s*\n\s*', ' ', app_name).strip())

                if not apps_ilimitadas_list_temp:
                    text_content_full = plan_element.get_text(separator=' ', strip=True)
                    match_apps_text = re.search(r'(?:Apps|Redes Sociales)\s+Ilimitadas(?::\s*(.*?))?(?=[.\n]|$)', text_content_full, re.IGNORECASE | re.DOTALL)
                    if match_apps_text:
                        if match_apps_text.group(1):
                            apps_ilimitadas_list_temp.extend([re.sub(r'\s*\n\s*', ' ', app.strip()).strip() for app in match_apps_text.group(1).split(',') if app.strip()])
                        else:
                            apps_ilimitadas_list_temp.append(re.sub(r'\s*\n\s*', ' ', match_apps_text.group(0).replace(":", "").strip()).strip())

                    keywords = ["WhatsApp", "Facebook", "Instagram", "TikTok", "Spotify", "Netflix", "Youtube", "Waze", "Twitter"]
                    for keyword in keywords:
                        if f"{keyword} Ilimitado" in text_content_full or f"Acceso ilimitado a {keyword}" in text_content_full:
                            if keyword.lower() not in [a.lower() for a in apps_ilimitadas_list_temp]:
                                apps_ilimitadas_list_temp.append(keyword)

                if apps_ilimitadas_list_temp:
                    apps_ilimitadas = ", ".join(sorted(list(set(apps_ilimitadas_list_temp))))

            minutos_llamadas = 'No especificado'
            sms = 'No especificado'

            all_benefit_texts_raw = []
            benefit_text_tags_specific = plan_element.find_all('p', class_='stefa-parrilla_blanco--body-texto')
            for tag in benefit_text_tags_specific:
                all_benefit_texts_raw.append(tag.get_text(strip=True))

            details_container = plan_element.find('div', class_='p-plan__slide__details')
            if details_container:
                general_benefit_tags = details_container.find_all(['li', 'p', 'span', 'div'], class_=lambda x: x and ('benefit-item' in x or 'feature-row' in x or 'text-benefit' in x or 'item-detail' in x or 'body-text' in x or 'plan-detail' in x))
                for tag in general_benefit_tags:
                    all_benefit_texts_raw.append(tag.get_text(strip=True))

            full_plan_text = plan_element.get_text(separator=' ', strip=True)
            all_benefit_texts_raw.append(full_plan_text)

            processed_texts = set()

            for raw_text in all_benefit_texts_raw:
                cleaned_text = re.sub(r'\s*\n\s*', ' ', raw_text).strip()
                if not cleaned_text or cleaned_text in processed_texts:
                    continue
                processed_texts.add(cleaned_text)

                text_lower = cleaned_text.lower()

                if 'llamadas ilimitadas' in text_lower:
                    minutos_llamadas = 'Llamadas ilimitadas'
                    match_usa_canada = re.search(r'(\d+)\s*minutos\s*(?:para|a)\s*(?:usa|eeuu)\s*(?:y|e)\s*canadá', text_lower)
                    if match_usa_canada:
                        minutos_llamadas += f", {match_usa_canada.group(1)} minutos para Usa y Canadá"
                elif re.search(r'(\d+)\s*minutos', text_lower):
                    match_minutes = re.search(r'(\d+)\s*minutos', text_lower)
                    if match_minutes:
                        minutos_llamadas = f"{match_minutes.group(1)} minutos"

                if 'sms ilimitados' in text_lower:
                    sms = 'SMS ilimitados'
                elif re.search(r'(\d+)\s*sms', text_lower):
                    match_sms = re.search(r'(\d+)\s*sms', text_lower)
                    if match_sms:
                        sms = f"{match_sms.group(1)} SMS"

            if "internet + llamadas ilimitadas" in apps_ilimitadas.lower() and minutos_llamadas == 'No especificado':
                minutos_llamadas = "Llamadas ilimitadas"
                apps_ilimitadas = apps_ilimitadas.replace("Internet + Llamadas Ilimitadas", "Internet Ilimitado (si aplica)").strip()
                if not apps_ilimitadas or apps_ilimitadas.lower() == "internet ilimitado (si aplica)".lower():
                    apps_ilimitadas = 'No especificado'

            planes_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Detalles de Gigas': detalles_gigas,
                'Apps Ilimitadas': apps_ilimitadas,
                'Minutos/Llamadas': minutos_llamadas,
                'SMS': sms,
                'Tipo de Contratación': tipo_contratacion,
                'Vigencia de Promoción': vigencia_promocion
            })

    except Exception as e:
        print(f"Ocurrió un error crítico durante la extracción de Movistar: {e}")
        return []
    finally:
        if driver:
            driver.quit()

    return planes_data

def extraer_planes_claro_colab():
    """
    Función para extraer planes de Claro.
    Este código usa Selenium y BeautifulSoup para el scraping.
    Retorna una lista de diccionarios con los datos de los planes en formato estandarizado.
    """
    url = "https://www.claro.com.pe/personas/movil/postpago/"
    plans_data = []
    processed_plans = set()

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--log-level=3')

    driver = None

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'cA1PEBodyCardWrap')))
        time.sleep(5)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        plan_elements = soup.find_all('div', class_='cA1PEBodyCardWrap')

        if not plan_elements:
            print("ERROR (Claro): No se encontraron elementos de planes en la página. La estructura HTML podría haber cambiado.")
            return []

        for plan_element in plan_elements:
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            detalles_gigas = 'N/A'
            apps_ilimitadas = 'No especificado'
            minutos_llamadas = 'No especificado'
            sms = 'No especificado'
            tipo_contratacion = 'Postpago'
            vigencia_promocion = 'N/A'

            name = plan_element.get('data-badge', 'N/A')
            nombre_plan = name

            price_str = plan_element.get('data-price', 'N/A')
            try:
                precio = float(price_str)
            except ValueError:
                precio = 'N/A'

            plan_key = (nombre_plan, precio)
            if plan_key in processed_plans:
                continue
            processed_plans.add(plan_key)

            gigas_tag = plan_element.find('span', class_='number')
            if gigas_tag:
                gigas_text = gigas_tag.get_text(strip=True)
                if gigas_text.upper().endswith('GB'):
                    gigas = gigas_text
                    if "ilimitado" in plan_element.get_text().lower() or "velocidad reducida" in plan_element.get_text().lower():
                        detalles_gigas = f"{gigas} en Alta Velocidad (luego velocidad reducida)"
                    else:
                        detalles_gigas = "En Alta Velocidad"
                elif "ilimitado" in gigas_text.lower():
                    gigas = "Ilimitados"
                    detalles_gigas = "Datos ilimitados (velocidad puede reducirse)"
                else:
                    gigas = gigas_text
                    detalles_gigas = "N/A"

            is_max_ilimitado_promo = False
            promo_div = plan_element.find('div', class_='cardPePromo')
            if promo_div:
                promo_text_span = promo_div.find('span', string=lambda text: text and 'Gigas, Minutos y SMS' in text)
                if promo_text_span:
                    is_max_ilimitado_promo = True
                    minutos_llamadas = "Ilimitadas"
                    sms = "Ilimitados"
                    apps_ilimitadas = "Incluidas en Todo Ilimitado"

            if not is_max_ilimitado_promo:
                apps_list = []
                app_icon_tags = plan_element.find_all('i', class_=lambda x: x and 'cIco-rs-' in x)
                for icon_tag in app_icon_tags:
                    for cls in icon_tag.get('class', []):
                        if 'cIco-rs-' in cls:
                            app_name = cls.replace('cIco-rs-', '')
                            apps_list.append(app_name.capitalize())
                apps_ilimitadas = ", ".join(apps_list) if apps_list else 'No especificado'
                if apps_list:
                    apps_ilimitadas = f"{apps_ilimitadas} ilimitadas"

                if minutos_llamadas == 'No especificado':
                    span_element_with_text = plan_element.find('span', string=lambda text: text and ('Llamadas y SMS' in text.strip() or 'Llamadas' in text.strip() or 'SMS' in text.strip()))
                    if span_element_with_text:
                        dt_parent = span_element_with_text.find_parent('dt')
                        if dt_parent:
                            dd_element = dt_parent.find_next_sibling('dd')
                            if dd_element:
                                calls_sms_text = dd_element.get_text(strip=True)
                                if "ilimitadas" in calls_sms_text.lower():
                                    minutos_llamadas = "Llamadas ilimitadas"
                                    sms = "SMS ilimitados"
                                else:
                                    match_minutes = re.search(r'(\d+)\s*minutos', calls_sms_text, re.IGNORECASE)
                                    if match_minutes:
                                        minutos_llamadas = f"{match_minutes.group(1)} minutos"
                                    else:
                                        minutos_llamadas = calls_sms_text
                                    match_sms = re.search(r'(\d+)\s*sms', calls_sms_text, re.IGNORECASE)
                                    if match_sms:
                                        sms = f"{match_sms.group(1)} SMS"
                                    else:
                                        sms = calls_sms_text

            plans_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Detalles de Gigas': detalles_gigas,
                'Apps Ilimitadas': apps_ilimitadas,
                'Minutos/Llamadas': minutos_llamadas,
                'SMS': sms,
                'Tipo de Contratacion': tipo_contratacion,
                'Vigencia de Promoción': vigencia_promocion
            })

    except Exception as e:
        print(f"Ocurrió un error crítico durante la extracción de Claro: {e}")
        return []
    finally:
        if driver:
            driver.quit()

    return plans_data

def extraer_planes_bitel_colab():
    """
    Función para extraer planes de Bitel.
    Este código usa Selenium y BeautifulSoup para el scraping.
    Retorna una lista de diccionarios con los datos de los planes.
    """
    url = "https://bitel.com.pe/planes/control/ilimitado"
    planes_data = []

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1990,1080')
    options.add_argument('--log-level=3')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = None
    wait = None

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        wait = WebDriverWait(driver, 45)

        try:
            common_close_xpaths = [
                "//button[contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'Cerrar') or contains(., 'OK')]",
                "//a[contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'Cerrar') or contains(., 'OK')]",
                "//div[contains(@class, 'close-button') or contains(@class, 'modal-close') or contains(@class, 'btn-close') or contains(@class, 'close-popup')]",
                "//span[contains(text(), 'x') or contains(text(), 'X') or @class='close-icon']",
                "//button[contains(@id, 'cookie') or contains(@id, 'modal') or contains(@class, 'cookie') or contains(@class, 'modal')][contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'OK')]",
                "//div[@role='dialog']//button[contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'OK')]"
            ]

            found_and_clicked = False
            for xpath_str in common_close_xpaths:
                try:
                    btn = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, xpath_str))
                    )
                    if btn.is_displayed() and btn.is_enabled():
                        btn.click()
                        time.sleep(2)
                        found_and_clicked = True
                        break
                except:
                    pass

        except Exception as e:
            pass

        try:
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'cont-package')))
        except Exception as e:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            try:
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'cont-package')))
            except Exception as e:
                print(f"ERROR FATAL: 'cont-package' no se hizo visible incluso después de scroll. {e}")
                print("La estructura de la página o la forma en que carga el contenido ha cambiado significativamente o hay un bloqueo persistente.")
                return []

        time.sleep(5)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        plan_elements = soup.find_all('div', class_='cont-package')

        if not plan_elements:
            print("ERROR: Después de la carga, no se encontraron elementos con la clase 'cont-package' en BeautifulSoup.")
            print("Esto podría indicar que la clase ha cambiado o el contenido no está en el HTML parseado.")
            return []

        for i, plan_element in enumerate(plan_elements):
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            apps_ilimitadas = 'No especificado'
            minutos_llamadas = 'No especificado'
            sms = 'No especificado'
            detalles_gigas = 'N/A'

            full_plan_text_raw = plan_element.get_text(separator=' ', strip=True)
            full_plan_text_lower = full_plan_text_raw.lower()

            match_precio = re.search(r's/\s*(\d+\.\d+)', full_plan_text_lower)
            if match_precio:
                precio = f"S/ {match_precio.group(1)}"
            else:
                price_element_a = plan_element.find('a', href="javascript:void(0);")
                if price_element_a:
                    title_attr = price_element_a.get('title')
                    if title_attr:
                        match_price_title = re.search(r'(\d+\.\d+)', title_attr)
                        if match_price_title:
                            precio = f"S/ {match_price_title.group(1)}"

                if precio == 'N/A':
                    match_precio_simple = re.search(r'(\d+\.\d+)', full_plan_text_lower)
                    if match_precio_simple:
                        precio_val = float(match_precio_simple.group(1))
                        if 10.00 <= precio_val <= 200.00:
                            precio = f"S/ {match_precio_simple.group(1)}"

            if precio != 'N/A':
                nombre_plan = f"Ilimitado - {precio.replace('S/ ', '')}"
            else:
                name_span_element = plan_element.find('span', class_='color-white text-bold title-1g')
                if name_span_element:
                    span_text = name_span_element.get_text(strip=True)
                    cleaned_span_text = re.sub(r'\s*S/\s*\d+\.\d+', '', span_text, flags=re.IGNORECASE).strip()
                    cleaned_span_text = re.sub(r'\s*\d+\.\d+$', '', cleaned_span_text, flags=re.IGNORECASE).strip()
                    if cleaned_span_text:
                        nombre_plan = f"Ilimitado - {cleaned_span_text}"
                    else:
                        nombre_plan = "Ilimitado - Precio No Disponible"
                else:
                    nombre_plan = "Ilimitado - Precio No Disponible"

            gigas_element = plan_element.find('p', class_='capa')

            if gigas_element:
                gigas_text = gigas_element.get_text(strip=True)
                gigas_text_lower = gigas_text.lower()

                period_element = plan_element.find('p', class_='period')
                period_text_lower = period_element.get_text(strip=True).lower() if period_element else ''

                if 'ilimitados' in gigas_text_lower:
                    gigas = 'Ilimitados'
                    match_gigas_alta_velocidad = re.search(r'(\d+)\s*GB(?:\s*en\s*alta\s*velocidad)?', gigas_text, re.IGNORECASE)
                    if match_gigas_alta_velocidad:
                        detalles_gigas = f"{match_gigas_alta_velocidad.group(1)} GB en Alta Velocidad"
                    elif 'alta velocidad' in period_text_lower:
                        detalles_gigas = "Velocidad reducida después de cierto consumo"
                    else:
                        detalles_gigas = "Datos ilimitados sin restricciones de velocidad explícitas"
                else:
                    match_gigas = re.search(r'(\d+)\s*GB', gigas_text, re.IGNORECASE)
                    if match_gigas:
                        gigas = f"{match_gigas.group(1)} GB"
                        if 'alta velocidad' in period_text_lower:
                            detalles_gigas = "En Alta Velocidad"
                    else:
                        gigas = gigas_text
                        if 'alta velocidad' in period_text_lower:
                            detalles_gigas = "En Alta Velocidad"
            else:
                gigas = 'N/A'
                detalles_gigas = 'Elemento de gigas no encontrado'

            app_keywords_patterns = {
                'WhatsApp': r'whatsapp ilimitado',
                'Facebook': r'facebook ilimitado',
                'Instagram': r'instagram ilimitado',
                'TikTok': r'tiktok ilimitado',
                'Spotify': r'spotify ilimitado',
                'Waze': r'waze ilimitado',
                'YouTube': r'youtube ilimitado',
                'Apps ilimitadas x meses': r'apps ilimitadas (x \d+ meses)?',
                'Internet + Llamadas ilimitadas': r'internet \+\s*llamadas ilimitadas'
            }
            found_apps = []
            for app_name, pattern in app_keywords_patterns.items():
                if re.search(pattern, full_plan_text_lower):
                    if 'x \d+ meses' in pattern and re.search(r'x \d+ meses', full_plan_text_lower):
                        promo_match = re.search(r'x \d+ meses', full_plan_text_lower).group(0)
                        found_apps.append(f"{app_name.replace(' x meses', '')} {promo_match}")
                    else:
                        found_apps.append(app_name)

            app_image_elements = plan_element.find_all('li', class_='app-item')
            for app_li in app_image_elements:
                img_tag = app_li.find('img')
                if img_tag and 'alt' in img_tag.attrs:
                    app_name_from_alt = img_tag['alt'].strip()
                    if app_name_from_alt and app_name_from_alt.lower() not in [app.lower() for app in found_apps]:
                        found_apps.append(app_name_from_alt)

            apps_ilimitadas = ", ".join(sorted(list(set(found_apps)))) if found_apps else 'No especificado'

            minutos_llamadas = "No especificado"
            sms = "No especificado"

            todo_ilimitado_element = plan_element.find('p', class_=re.compile(r'title.*Todo ilimitado'))
            if todo_ilimitado_element and "todo ilimitado" in todo_ilimitado_element.get_text(strip=True).lower():
                minutos_llamadas = 'Llamadas ilimitadas'
                sms = 'SMS ilimitados'
            else:
                if 'llamadas ilimitadas perú' in full_plan_text_lower:
                    minutos_llamadas = 'Llamadas ilimitadas Perú'
                    match_usa_canada = re.search(r'(\d+)\s*minutos\s*(?:para|a)\s*(?:usa|eeuu)\s*(?:y|e)\s*canadá', full_plan_text_lower)
                    if match_usa_canada:
                        minutos_llamadas += f", {match_usa_canada.group(1)} minutos para Usa y Canadá"
                elif 'llamadas ilimitadas' in full_plan_text_lower:
                    minutos_llamadas = 'Llamadas ilimitadas'

                if 'sms ilimitados' in full_plan_text_lower:
                    sms = 'SMS ilimitados'
                else:
                    match_sms = re.search(r'(\d+)\s*sms', full_plan_text_lower)
                    if match_sms:
                        sms = f"{match_sms.group(1)} SMS"

            benefit_period_element = plan_element.find('p', class_='benefit-period')
            if benefit_period_element and "internet, llamadas y sms" in benefit_period_element.get_text(strip=True).lower():
                if minutos_llamadas == "No especificado":
                    minutos_llamadas = 'Llamadas incluidas'
                if sms == "No especificado":
                    sms = 'SMS incluidos'

            planes_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Detalles de Gigas': detalles_gigas,
                'Apps Ilimitadas': apps_ilimitadas,
                'Minutos/Llamadas': minutos_llamadas,
                'SMS': sms,
                'Tipo de Contratación': 'Postpago',
                'Vigencia de Promoción': 'N/A'
            })

    except Exception as e:
        print(f"Ocurrió un error crítico durante la ejecución: {e}")
        return []
    finally:
        if driver:
            driver.quit()

    return planes_data

def extraer_planes_entel_colab():
    """
    Función para extraer planes de Entel.
    Este código usa requests y BeautifulSoup para el scraping.
    Retorna una lista de diccionarios con los datos de los planes en formato estandarizado.
    """
    url = "https://ofertasentel.pe/?utm_source=bing&utm_medium=cpc_search&utm_campaign=pospago_promo_marcaplanes&utm_term=marca_planes&utmcampaign=0104020302&msclkid=dfab6aa18cde18a17c3b02cd920ff43b"
    planes_data = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        plan_boxes = soup.find_all('div', class_='box')

        if not plan_boxes:
            print("ERROR (Entel): No se encontraron elementos con la clase 'box' en BeautifulSoup.")
            return []

        for box in plan_boxes:
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            detalles_gigas = 'N/A'
            apps_ilimitadas = 'No especificado'
            minutos_llamadas = 'No especificado'
            sms = 'No especificado'
            tipo_contratacion = 'Postpago'
            vigencia_promocion = 'N/A'

            plan_name_tag = box.find('div', class_='property property_internet')
            if plan_name_tag and plan_name_tag.find('b'):
                nombre_plan = plan_name_tag.find('b').get_text(strip=True)
            elif box.find('h3'):
                nombre_plan = box.find('h3').get_text(strip=True)

            price_tag = box.find('div', class_='price')
            if price_tag:
                precio = price_tag.get_text(strip=True)

            gb_div = box.find('div', class_='gb')
            if gb_div:
                strong_text = gb_div.find('strong').get_text(strip=True) if gb_div.find('strong') else ''
                span_text = gb_div.find('span').get_text(strip=True) if gb_div.find('span') else ''

                if 'ilimitado' in strong_text.lower():
                    gigas = 'Ilimitados'
                    if span_text:
                        detalles_gigas = span_text.strip()
                    else:
                        detalles_gigas = "Datos ilimitados sin restricciones de velocidad explícitas"
                else:
                    gigas = strong_text.strip()
                    if span_text:
                        detalles_gigas = span_text.strip()
                    else:
                        detalles_gigas = "N/A"

            app_tag = box.find('div', class_='property property_app')
            if app_tag:
                app_list_items = app_tag.find_all('li')
                if app_list_items:
                    apps_ilimitadas = ", ".join(sorted(list(set([li.get_text(strip=True) for li in app_list_items if li.get_text(strip=True)]))))
                else:
                    apps_ilimitadas = app_tag.get_text(strip=True, separator=' ')
                if "ilimitado" not in apps_ilimitadas.lower() and "ilimitadas" not in apps_ilimitadas.lower() and apps_ilimitadas != 'N/A' and apps_ilimitadas != 'No especificado':
                     apps_ilimitadas = f"{apps_ilimitadas} ilimitadas"

            llamadas_tag = box.find('div', class_='property property_llamadas')
            if llamadas_tag:
                llamadas_text = llamadas_tag.get_text(strip=True)
                if "ilimitadas" in llamadas_text.lower():
                    minutos_llamadas = "Llamadas ilimitadas"
                else:
                    minutos_llamadas = llamadas_text

            sms_tag = box.find('div', class_='property property_sms')
            if sms_tag:
                sms_text = sms_tag.get_text(strip=True)
                if "ilimitados" in sms_text.lower():
                    sms = "SMS ilimitados"
                else:
                    sms = sms_text

            planes_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Detalles de Gigas': detalles_gigas,
                'Apps Ilimitadas': apps_ilimitadas,
                'Minutos/Llamadas': minutos_llamadas,
                'SMS': sms,
                'Tipo de Contratacion': tipo_contratacion,
                'Vigencia de Promoción': vigencia_promocion
            })

    except requests.exceptions.RequestException as e:
        print(f"Error de red al acceder a la página de Entel: {e}")
        return []
    except Exception as e:
        print(f"Ocurrió un error inesperado durante el scraping de Entel: {e}")
        return []

    return planes_data

def standardize_plan_data(plan):
    """
    Estandariza las claves y los tipos de datos de un diccionario de plan.
    Asegura que 'Precio (S/)' sea un float.
    """
    # Definir todas las claves esperadas y sus valores por defecto
    standardized_plan = {
        'Operador': plan.get('Operador', 'N/A'),
        'Nombre del Plan': plan.get('Nombre del Plan', 'N/A'),
        'Precio (S/)': plan.get('Precio (S/)', 'N/A'),
        'Gigas': plan.get('Gigas', 'N/A'),
        'Detalles de Gigas': plan.get('Detalles de Gigas', 'N/A'),
        'Apps Ilimitadas': plan.get('Apps Ilimitadas', 'No especificado'),
        'Minutos/Llamadas': plan.get('Minutos/Llamadas', 'No especificado'),
        'SMS': plan.get('SMS', 'No especificado'),
        'Tipo de Contratación': plan.get('Tipo de Contratación', 'Postpago'),
        'Vigencia de Promoción': plan.get('Vigencia de Promoción', 'N/A')
    }

    # Convertir Precio (S/) a float
    precio_raw = standardized_plan['Precio (S/)']
    if isinstance(precio_raw, str):
        precio_str = precio_raw.replace('S/', '').replace(',', '.').strip()
        try:
            standardized_plan['Precio (S/)'] = float(precio_str)
        except ValueError:
            standardized_plan['Precio (S/)'] = None # Usar None si no se puede convertir a número
    elif not isinstance(precio_raw, (int, float)):
        standardized_plan['Precio (S/)'] = None # Asegurarse de que sea None si no es número ni string convertible

    return standardized_plan

def get_all_telecom_plans():
    """
    Extrae y combina los planes de todos los operadores.
    Retorna un DataFrame de Pandas con los datos estandarizados.
    """
    all_plans = []

    # --- Extraer y combinar planes de Movistar ---
    try:
        movistar_plans = extraer_planes_movistar_colab()
        for plan in movistar_plans:
            plan['Operador'] = 'Movistar'
            all_plans.append(standardize_plan_data(plan))
    except Exception as e:
        print(f"Error al extraer planes de Movistar: {e}")

    # --- Extraer y combinar planes de Claro ---
    try:
        claro_plans = extraer_planes_claro_colab()
        for plan in claro_plans:
            plan['Operador'] = 'Claro'
            all_plans.append(standardize_plan_data(plan))
    except Exception as e:
        print(f"Error al extraer planes de Claro: {e}")

    # --- Extraer y combinar planes de Bitel ---
    try:
        bitel_plans = extraer_planes_bitel_colab()
        for plan in bitel_plans:
            plan['Operador'] = 'Bitel'
            all_plans.append(standardize_plan_data(plan))
    except Exception as e:
        print(f"Error al extraer planes de Bitel: {e}")

    # --- Extraer y combinar planes de Entel ---
    try:
        entel_plans = extraer_planes_entel_colab()
        for plan in entel_plans:
            plan['Operador'] = 'Entel'
            all_plans.append(standardize_plan_data(plan))
    except Exception as e:
        print(f"Error al extraer planes de Entel: {e}")

    df = pd.DataFrame(all_plans)

    # Ordenar por precio, asegurando que los valores 'N/A' o None vayan al final
    df['Precio_Sort'] = df['Precio (S/)'].apply(lambda x: x if isinstance(x, (int, float)) else float('inf'))
    df_sorted = df.sort_values(by='Precio_Sort').drop(columns='Precio_Sort')

    return df_sorted

# --- Inicializar la aplicación Dash ---
app = dash.Dash(__name__)

# Cargar los datos iniciales
df_plans = get_all_telecom_plans()

# Obtener opciones únicas de operadores para el dropdown
operator_options = [{'label': 'Todos', 'value': 'all'}] + \
                   [{'label': op, 'value': op} for op in df_plans['Operador'].unique()]

# --- Diseño de la aplicación Dash ---
app.layout = html.Div(style={'fontFamily': 'Arial, sans-serif', 'padding': '20px', 'backgroundColor': '#f4f4f4', 'color': '#333'}, children=[
    html.H1("Comparador de Planes Telefónicos", style={'textAlign': 'center', 'color': '#007bff'}),

    html.Div([
        html.Label("Filtrar por Operador:", style={'marginRight': '10px', 'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='operator-dropdown',
            options=operator_options,
            value='all',
            clearable=False,
            style={'width': '200px', 'display': 'inline-block', 'verticalAlign': 'middle'}
        ),
    ], style={'marginBottom': '20px', 'textAlign': 'center'}),

    dash_table.DataTable(
        id='plans-table',
        columns=[{"name": i, "id": i} for i in df_plans.columns],
        data=df_plans.to_dict('records'),
        style_table={'overflowX': 'auto', 'boxShadow': '0 4px 12px rgba(0,0,0,0.15)', 'borderRadius': '8px'},
        style_header={
            'backgroundColor': '#00AEEF',
            'color': 'white',
            'fontWeight': 'bold',
            'textAlign': 'left',
            'padding': '15px 20px',
            'textTransform': 'uppercase',
            'fontSize': '0.95em',
            'letterSpacing': '0.5px'
        },
        style_data={
            'backgroundColor': 'white',
            'color': '#333',
            'borderBottom': '1px solid #eee',
            'padding': '15px 20px'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'even'},
                'backgroundColor': '#f8f8f8'
            },
            {
                'if': {'state': 'active'},
                'backgroundColor': '#e0f7fa',
                'border': '1px solid #00AEEF'
            }
        ],
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_size=10,
    )
])

# --- Callbacks para interactividad ---
@app.callback(
    Output('plans-table', 'data'),
    Input('operator-dropdown', 'value')
)
def update_table(selected_operator):
    if selected_operator == 'all':
        return df_plans.to_dict('records')
    else:
        filtered_df = df_plans[df_plans['Operador'] == selected_operator]
        return filtered_df.to_dict('records')

# --- Ejecutar la aplicación Dash en Colab ---
if __name__ == '__main__':
    app.run(mode='inline', port=8050)


gpg: cannot open '/dev/tty': No such device or address
deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main
Hit:1 http://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 128 kB in 1s (97.0 kB/s

<IPython.core.display.Javascript object>